![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# Classificação de Atividade Humana com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix


filename_features = r"C:\Users\cepir\Documents\Henrique\EBAC\features.txt"
filename_labels = r"C:\Users\cepir\Documents\Henrique\EBAC\activity_labels.txt"

filename_subtrain = r"C:\Users\cepir\Documents\Henrique\EBAC\subject_train.txt"
filename_xtrain = r"C:\Users\cepir\Documents\Henrique\EBAC\X_train.txt"
filename_ytrain = r"C:\Users\cepir\Documents\Henrique\EBAC\y_train.txt"

filename_subtest = r"C:\Users\cepir\Documents\Henrique\EBAC\subject_test.txt"
filename_xtest = r"C:\Users\cepir\Documents\Henrique\EBAC\X_test.txt"
filename_ytest = r"C:\Users\cepir\Documents\Henrique\EBAC\y_test.txt"

# Carrega os nomes das features
features = pd.read_csv(filename_features, header=None, names=['cod_label', 'nome_var'], sep="#")
labels = pd.read_csv(filename_labels, sep="\s+", header=None, names=['cod_label', 'label'])

# Corrigido: usa apenas a coluna com os nomes das variáveis
feature_names = features['nome_var'].tolist()

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])
X_train = pd.read_csv(filename_xtrain, sep="\s+", header=None, names=feature_names)
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'])
X_test = pd.read_csv(filename_xtest, sep="\s+", header=None, names=feature_names)
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

## PCA com variáveis padronizadas

Reflexão sobre a escala das variáveis:

**Variáveis em métricas muito diferentes** podem interferir na análise de componentes principais. Lembra que variância é informação pra nós? Pois bem, tipicamente se há uma variável monetária como salário, vai ter uma ordem de variabilidade bem maior que número de filhos, tempo de emprego ou qualquer variável dummy. Assim, as variáveis de maior variância tendem a "dominar" a análise. Nesses casos é comum usar a padronização das variáveis.

Faça duas análises de componentes principais para a base do HAR - com e sem padronização e compare:

- A variância explicada por componente
- A variância explicada acumulada por componente
- A variância percentual por componente
- A variância percentual acumulada por componente
- Quantas componentes você escolheria, em cada caso para explicar 90% da variância?

In [3]:
%%time

def padroniza(s):
    if s.std() > 0:
        s = (s - s.mean())/s.std()
    return s

X_train_pad = pd.DataFrame(X_train).apply(padroniza, axis=0)
X_train_pad.head()

CPU times: total: 656 ms
Wall time: 828 ms


,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0.200628,-0.063678,-0.419600,-0.868755,-0.939377,-0.737479,-0.859758,-0.938955,-0.766385,-0.855978,...,-0.795305,0.025958,-0.276380,-0.360579,0.062935,-0.778374,-0.026079,-0.687172,0.407918,-0.007567
1,0.055944,0.031484,-0.253891,-0.875366,-0.923839,-0.849247,-0.868472,-0.921936,-0.848870,-0.871300,...,0.130605,-0.897296,-0.767938,0.133002,-0.021460,-1.218722,1.484369,-0.694091,0.409089,0.007875
2,0.073510,-0.043414,-0.076289,-0.868980,-0.907698,-0.893724,-0.863078,-0.898793,-0.896640,-0.863264,...,1.152257,-0.260860,-0.438286,-0.377815,0.391949,0.151197,1.704085,-0.702191,0.410260,0.026501
3,0.066691,-0.208407,-0.249695,-0.870566,-0.939959,-0.921743,-0.864445,-0.938060,-0.925216,-0.863264,...,1.112694,0.591005,0.463123,-0.135016,-0.033635,1.037781,-1.002951,-0.701636,0.414622,0.031712
4,0.030467,0.027585,-0.109840,-0.875128,-0.934815,-0.921281,-0.867325,-0.931726,-0.927965,-0.870201,...,-0.149567,-0.138505,-0.240296,0.340383,0.268468,1.125841,-1.276196,-0.700104,0.425434,0.045222


In [6]:
# com padronização

pca = PCA()
pca.fit(X_train)

# variância explicada por componente
var_exp = pca.explained_variance_ratio_

# variância acumulada por componente
var_exp_acum = np.cumsum(var_exp)

# variância percentual por componente
var_pct = var_exp * 100

# variância percentual acumulada por componente
var_pct_acum = var_exp_acum * 100

# número de componentes para explicar 90%
n_components_90 = np.argmax(var_exp_acum >= 0.9) + 1

Sem padronização:
Variância percentual acumulada: [ 62.55443998  67.4674627   71.58893016  73.46388628  75.15874627
  76.43081555  77.60750069  78.67647386  79.64585363  80.50387181
  81.26617372  81.93861938  82.51803897  83.07591961  83.57484534
  84.04978297  84.51698308  84.94860094  85.37431612  85.78471116
  86.17871356  86.55402287  86.90645036  87.24580979  87.57794878
  87.89737757  88.19915672  88.49093929  88.78050925  89.06243704
  89.33914119  89.60253624  89.85784293  90.09370881  90.32436112
  90.54800929  90.77095742  90.9812334   91.18962632  91.39440007
  91.58725653  91.77613615  91.95731641  92.13678911  92.30911678
  92.46931872  92.62635821  92.78298558  92.93595543  93.08630671
  93.23142442  93.37206458  93.50888964  93.63574755  93.76075366
  93.88049584  93.99861565  94.11361054  94.22669303  94.33636258
  94.44406665  94.54896693  94.65286105  94.75433544  94.85348714
  94.95051214  95.04592206  95.13738453  95.22615131  95.31375173
  95.39902494  95.48192919

In [7]:
# com padronização

pca_pad = PCA()
pca_pad.fit(X_train_pad)

var_exp_pad = pca_pad.explained_variance_ratio_
var_exp_acum_pad = np.cumsum(var_exp_pad)
var_pct_pad = var_exp_pad * 100
var_pct_acum_pad = var_exp_acum_pad * 100
n_components_90_pad = np.argmax(var_exp_acum_pad >= 0.9) + 1

Com padronização:
Variância percentual acumulada: [ 50.78117229  57.36185256  60.16828933  62.67224208  64.56052709
  66.28453351  67.65554498  68.85462266  69.85048217  70.81556876
  71.67562041  72.47590136  73.23989773  73.88522665  74.517551
  75.11727309  75.70402339  76.27943078  76.84735183  77.37464761
  77.87501053  78.36341894  78.84162472  79.31018765  79.75947691
  80.18050415  80.59848284  81.00405321  81.39257737  81.77959542
  82.1455543   82.50010768  82.84805028  83.18523739  83.51491439
  83.84312944  84.16365892  84.45927386  84.74599627  85.03107082
  85.29983714  85.565457    85.82886299  86.08771359  86.33676846
  86.58372249  86.82440241  87.06051748  87.29079634  87.51836358
  87.73852828  87.95199527  88.15969972  88.36219634  88.56197578
  88.75972641  88.95400004  89.1442372   89.33230209  89.51851806
  89.6999846   89.87736559  90.05345066  90.22673457  90.39747289
  90.56602096  90.73163589  90.89473834  91.05317512  91.21067266
  91.36369556  91.51590081  

In [8]:
print("SEM padronização:")
print("1. Variância explicada por componente: \n", var_exp)
print("2. Variância explicada acumulada por componente: \n", var_exp_acum)
print("3. Variância percentual por componente (%): \n", var_pct)
print("4. Variância percentual acumulada por componente (%): \n", var_pct_acum)
print("5. Componentes para ≥ 90% da variância: ", n_components_90, "\n")

SEM padronização:
1. Variância explicada por componente: 
 [6.25544400e-01 4.91302272e-02 4.12146745e-02 1.87495612e-02
 1.69485999e-02 1.27206929e-02 1.17668514e-02 1.06897316e-02
 9.69379779e-03 8.58018171e-03 7.62301911e-03 6.72445661e-03
 5.79419596e-03 5.57880638e-03 4.98925730e-03 4.74937626e-03
 4.67200109e-03 4.31617860e-03 4.25715180e-03 4.10395039e-03
 3.94002401e-03 3.75309316e-03 3.52427488e-03 3.39359425e-03
 3.32138998e-03 3.19428786e-03 3.01779149e-03 2.91782575e-03
 2.89569956e-03 2.81927795e-03 2.76704148e-03 2.63395052e-03
 2.55306687e-03 2.35865883e-03 2.30652304e-03 2.23648168e-03
 2.22948133e-03 2.10275984e-03 2.08392917e-03 2.04773752e-03
 1.92856457e-03 1.88879623e-03 1.81180259e-03 1.79472696e-03
 1.72327672e-03 1.60201939e-03 1.57039496e-03 1.56627367e-03
 1.52969849e-03 1.50351278e-03 1.45117716e-03 1.40640152e-03
 1.36825069e-03 1.26857902e-03 1.25006115e-03 1.19742181e-03
 1.18119807e-03 1.14994893e-03 1.13082490e-03 1.09669551e-03
 1.07704072e-03 1.04900274

In [9]:
print("COM padronização:")
print("1. Variância explicada por componente: \n", var_exp_pad)
print("2. Variância explicada acumulada por componente: \n", var_exp_acum_pad)
print("3. Variância percentual por componente (%): \n", var_pct_pad)
print("4. Variância percentual acumulada por componente (%): \n", var_pct_acum_pad)
print("5. Componentes para ≥ 90% da variância: ", n_components_90_pad)

COM padronização:
1. Variância explicada por componente: 
 [5.07811723e-01 6.58068027e-02 2.80643677e-02 2.50395275e-02
 1.88828501e-02 1.72400642e-02 1.37101147e-02 1.19907768e-02
 9.95859511e-03 9.65086590e-03 8.60051649e-03 8.00280950e-03
 7.63996369e-03 6.45328924e-03 6.32324346e-03 5.99722090e-03
 5.86750302e-03 5.75407392e-03 5.67921047e-03 5.27295779e-03
 5.00362925e-03 4.88408406e-03 4.78205784e-03 4.68562930e-03
 4.49289260e-03 4.21027243e-03 4.17978682e-03 4.05570369e-03
 3.88524168e-03 3.87018044e-03 3.65958878e-03 3.54553381e-03
 3.47942606e-03 3.37187109e-03 3.29676999e-03 3.28215050e-03
 3.20529478e-03 2.95614939e-03 2.86722415e-03 2.85074548e-03
 2.68766324e-03 2.65619859e-03 2.63405989e-03 2.58850596e-03
 2.49054872e-03 2.46954034e-03 2.40679913e-03 2.36115075e-03
 2.30278857e-03 2.27567245e-03 2.20164699e-03 2.13466984e-03
 2.07704456e-03 2.02496620e-03 1.99779441e-03 1.97750625e-03
 1.94273627e-03 1.90237160e-03 1.88064891e-03 1.86215970e-03
 1.81466544e-03 1.77380987

## Árvore com PCA

Faça duas uma árvore de decisão com 10 componentes principais - uma com base em dados padronizados e outra sem padronizar. Utilize o ```ccp_alpha=0.001```.

Compare a acurácia na base de treino e teste.

In [11]:
%%time
# sem padronização
pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

clf = DecisionTreeClassifier(ccp_alpha=0.001)
clf.fit(X_train_pca, y_train)
acc_train = accuracy_score(y_train, clf.predict(X_train_pca))
acc_test = accuracy_score(y_test, clf.predict(X_test_pca))

print("SEM padronização:")
print("Acurácia treino:", acc_train)
print("Acurácia teste:", acc_test)

SEM padronização:
Acurácia treino: 0.8926822633297062
Acurácia teste: 0.8238887003732609
CPU times: total: 922 ms
Wall time: 782 ms


In [13]:
%%time
# com padronização
X_test_pad = pd.DataFrame(X_test).apply(padroniza, axis=0)
pca_pad = PCA(n_components=10)
X_train_pad_pca = pca_pad.fit_transform(X_train_pad)
X_test_pad_pca = pca_pad.transform(X_test_pad)

clf_pad = DecisionTreeClassifier(ccp_alpha=0.001)
clf_pad.fit(X_train_pad_pca, y_train)
acc_train_pad = accuracy_score(y_train, clf_pad.predict(X_train_pad_pca))
acc_test_pad = accuracy_score(y_test, clf_pad.predict(X_test_pad_pca))

print("\nCOM padronização:")
print("Acurácia treino:", acc_train_pad)
print("Acurácia teste:", acc_test_pad)


COM padronização:
Acurácia treino: 0.8586779107725789
Acurácia teste: 0.7736681370885646
CPU times: total: 1.44 s
Wall time: 1.08 s


A acurácia diminuiu em ambos os casos. Isso se dá porque o PCA é ótimo para simplificação, visualização e evitar overfitting, mas pode reduzir a performance de modelos, dependendo de como a informação está distribuída nos componentes escolhidos.